In [ ]:
# pip install beautifulsoup4 lxml


  Using cached beautifulsoup4-4.13.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached lxml-6.0.1-cp313-cp313-win_amd64.whl.metadata (3.9 kB)
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.5-py3-none-any.whl (105 kB)
Using cached lxml-6.0.1-cp313-cp313-win_amd64.whl (4.0 MB)
Using cached soupsieve-2.8-py3-none-any.whl (36 kB)

   ---------------------------------------- 0/3 [soupsieve]
   ---------------------------------------- 0/3 [soupsieve]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   --

In [ ]:
# !pip install elasticsearch

In [14]:
from bs4 import BeautifulSoup
import html as htmllib
import re

def extract_plain_text(fields):
    """
    fields = article.get("fields", {}) from Guardian API result.
    Returns plain cleaned text or None.
    """
    if not fields:
        return None


    # fallback to HTML body
    body_html = fields.get("body")
    if not body_html:
        return None

    soup = BeautifulSoup(body_html, "lxml")

    # remove unwanted tags (figures, scripts, asides, ads, nav, footer)
    for tag in soup(["script", "style", "aside", "figure", "iframe", "header", "footer", "nav"]):
        tag.decompose()


    # get text with paragraph spacing
    text = soup.get_text(separator=" ")
    text = htmllib.unescape(text)

    # collapse multiple newlines and whitespace
    text = re.sub(
    r"(?im)^\s*(Updated\s+at\s+)?\d{1,2}\.\d{2}\s*(am|pm)\s+BST\s*$", 
    "", 
    text)
    text = re.sub(r'\n\s*\n+', '\n\n', text)   # keep paragraph breaks
    lines = [ln.strip() for ln in text.splitlines()]
    clean = "\n".join([ln for ln in lines if ln])

    # optional: trim extremely long articles (e.g., keep first N chars)
    # clean = clean[:20000]

    return clean


In [ ]:
import requests
import csv
from time import sleep

API_KEY = "022783ad-6e46-4ba2-af9a-3a7464467d02"
BASE_URL = "https://content.guardianapis.com/search"
PAGE_SIZE = 200  # max 200

def crawl_guardian(pages=1, output_file="guardian_articles.csv"):
    all_articles = []

    # open CSV for writing
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)  # quote all fields
        writer.writerow(["id","title","url","section","published_at","trail_text","body"])

        for page in range(1, pages + 1):
            params = {
                "api-key": API_KEY,
                "page": page,
                "page-size": PAGE_SIZE,
                "show-fields": "trailText,body",
                "order-by": "newest"
            }
            resp = requests.get(BASE_URL, params=params)
            data = resp.json()

            results = data.get("response", {}).get("results", [])
            if not results:
                break

            for article in results:
                fields = article.get("fields", {})
                body = extract_plain_text(fields)
                row = [
                    article.get("id", ""),
                    article.get("webTitle", ""),
                    article.get("webUrl", ""),
                    article.get("sectionName", ""),
                    article.get("webPublicationDate", ""),
                    fields.get("trailText", ""),
                    body or ""
                ]
                writer.writerow(row)
                all_articles.append(row)

            print(f"Page {page} done, got {len(results)} articles.")
            sleep(1)  # polite delay

    print(f"Saved {len(all_articles)} articles to {output_file}")
    return all_articles


Page 1 done, got 200 articles.
Page 2 done, got 200 articles.
Page 3 done, got 200 articles.
Page 4 done, got 200 articles.
Page 5 done, got 200 articles.
Page 6 done, got 200 articles.
Page 7 done, got 200 articles.
Page 8 done, got 200 articles.
Page 9 done, got 200 articles.
Page 10 done, got 200 articles.
Page 11 done, got 200 articles.
Page 12 done, got 200 articles.
Page 13 done, got 200 articles.
Page 14 done, got 200 articles.
Page 15 done, got 200 articles.
Page 16 done, got 200 articles.
Page 17 done, got 200 articles.
Page 18 done, got 200 articles.
Page 19 done, got 200 articles.
Page 20 done, got 200 articles.
Page 21 done, got 200 articles.
Page 22 done, got 200 articles.
Page 23 done, got 200 articles.
Page 24 done, got 200 articles.
Page 25 done, got 200 articles.
Page 26 done, got 200 articles.
Page 27 done, got 200 articles.
Page 28 done, got 200 articles.
Page 29 done, got 200 articles.
Page 30 done, got 200 articles.
Saved 6000 articles to guardian_articles.csv


In [ ]:
all_articles = crawl_guardian(pages=30)  # adjust number of pages
cols = ["id","title","url","section","published_at","trail_text","body"]
all_articles = [dict(zip(cols, row)) for row in all_articles]

In [16]:
import pickle
with open("D:/DH/Senior/Paperboy/src/pickled_data/raw_crawl.pkl", "wb") as f:
    pickle.dump(all_articles, f)  
print(f"Pickled {len(all_articles)} stories.")

Pickled 6000 stories.


In [ ]:
import sys, time, random
sys.path.append("D:/DH/Senior/Paperboy")  
from src.api.BGEM3 import embed_bgem3

count = 0
failed = []

for i, doc in enumerate(all_articles):
    text = f"{doc[1]}\n\n{doc[5]}"
    if len(text) > 4000:
        text = text[:4000]

    while True:  # keep retrying until success
        try:
            embedded = embed_bgem3(text)
            data = embedded.get("data", None)

            if data:
                doc["embedding"] = data[0]  # add embedding
                print(f"✅ Embedded article {i+1}/{len(all_articles)}")
                break  # success, move on
            else:
                print(f"⚠️ Empty response for article {i}, retrying...")
            
        except Exception as e:
            print(f"❌ Error on article {i}: {e}. Retrying...")

        # exponential backoff with jitter
        sleep_time = min(60, 2 ** min(6, count)) + random.random()
        time.sleep(sleep_time)

    count += 1

print(f"\nFinished. Embedded {count} articles. Failed: {failed}")



✅ Embedded article 1/6000
✅ Embedded article 2/6000
✅ Embedded article 3/6000
✅ Embedded article 4/6000
✅ Embedded article 5/6000
✅ Embedded article 6/6000
✅ Embedded article 7/6000
✅ Embedded article 8/6000
✅ Embedded article 9/6000
✅ Embedded article 10/6000
✅ Embedded article 11/6000
✅ Embedded article 12/6000
✅ Embedded article 13/6000
✅ Embedded article 14/6000
✅ Embedded article 15/6000
✅ Embedded article 16/6000
✅ Embedded article 17/6000
✅ Embedded article 18/6000
✅ Embedded article 19/6000
✅ Embedded article 20/6000
✅ Embedded article 21/6000
✅ Embedded article 22/6000
✅ Embedded article 23/6000
✅ Embedded article 24/6000
✅ Embedded article 25/6000
✅ Embedded article 26/6000
✅ Embedded article 27/6000
✅ Embedded article 28/6000
✅ Embedded article 29/6000
✅ Embedded article 30/6000
✅ Embedded article 31/6000
✅ Embedded article 32/6000
✅ Embedded article 33/6000
✅ Embedded article 34/6000
✅ Embedded article 35/6000
✅ Embedded article 36/6000
✅ Embedded article 37/6000
✅ Embedded

In [ ]:
import pandas as pd

# Show first 5 rows
row = all_articles[666]
# print(f"Article {i+1}")
print("Title:", row[1])
print("URL:", row[2])
print("Section:", row[3])
print("Published at:", row[4])
print("Trail text:", row[5])
print("body:", row[6])
print("embedding:", row[7])

In [3]:
print(all_articles[424])

['film/2025/oct/01/play-dirty-shane-black-review', 'Play Dirty review – Shane Black’s action comedy comeback is a blast', 'https://www.theguardian.com/film/2025/oct/01/play-dirty-shane-black-review', 'Film', '2025-10-01T07:01:05Z', 'The writer-director returns to the genre he knows best with a breezy winner led by Mark Wahlberg and a standout LaKeith Stanfield', 'Right around the time that Netflix changed the game by creating its own original movies, and then inspired other streamers to do the same, Shane Black released his last great action comedy  The Nice Guys  on the big screen. It was one of the writer-director’s best, as exciting as it was amusing, but the ecosystem had changed since his 90s heyday, and a wide audience didn’t materialise, the film one of the year’s more frustratingly underseen flops.     In the years since, the big, star-led action comedy has become streaming bread and butter, most weeks cursed by another limp, murkily shot attempt to recall the joys of Black’s g

In [4]:
for i, row in enumerate(all_articles[:3]):  # just peek at first 3
    print(i, len(row), type(row[-1]))
    
print(all_articles[0])


0 8 <class 'list'>
1 8 <class 'list'>
2 8 <class 'list'>
['australia-news/live/2025/oct/03/png-agreement-aukus-submarines-pact-anthony-albanese-richard-marles-retired-admiral-slams-aukus-deal-gaza-flotilla-intercepted-ntwnfb', 'Australia news live: pro-Palestine Opera House protest could go to supreme court; AEC takes legal action against man allegedly behind unauthorised Wentworth pamphlets', 'https://www.theguardian.com/australia-news/live/2025/oct/03/png-agreement-aukus-submarines-pact-anthony-albanese-richard-marles-retired-admiral-slams-aukus-deal-gaza-flotilla-intercepted-ntwnfb', 'Australia news', '2025-10-03T04:23:58Z', 'Follow the latest news live', "5.23am  BST       South-west Sydney operation concludes after massive police response   Returning to the police operation in south-west Sydney this morning,  NSW police  have now confirmed it has concluded.    In a statement, a spokesperson said police were called to a house on Wollongbar Avenue in Panania just before 8am today, f

In [59]:
import psycopg2

def get_connection():
    return psycopg2.connect(
        host="localhost",
        port=5434,
        dbname="paperboy",
        user="minhhieu",
        password="minhhieu888"
    )

# UPSERT function
def upsert_articles(all_articles):
    sql = """
    INSERT INTO articles (id, title, url, section, published_at, trail_text, body, embedding)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (id) DO NOTHING
    """

    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.executemany(sql, all_articles)  # batch insert
        conn.commit()
    print(f"Upserted {len(all_articles)} articles.")


In [60]:
upsert_articles(all_articles)

Upserted 6000 articles.


In [25]:
empty_text_count = 0
empty_full_text_count = 0

for article in all_articles:
    if article[5] == None:
        empty_text_count += 1
    if article[1] == "":
        empty_full_text_count += 1

total = len(df)
print(f"Total articles: {total}")
print(f"Articles with empty 'text': {empty_text_count} ({empty_text_count/total*100:.2f}%)")
print(f"Articles with empty 'full_text': {empty_full_text_count} ({empty_full_text_count/total*100:.2f}%)")

Total articles: 6000
Articles with empty 'text': 0 (0.00%)
Articles with empty 'full_text': 0 (0.00%)


In [5]:
from pathlib import Path
import pickle

ROOT = Path("d:/DH/Senior/Paperboy/src")  # your project root
file_path = ROOT / "pickled_data" / "articles.pkl"
    
with open(file_path, "rb") as f:
    all_articles = pickle.load(f)

print("Loaded:", len(all_articles))

Loaded: 6000


In [2]:
print(all_articles[666])

['technology/2025/sep/29/waymo-illegal-u-turn-driverless-car', 'California police stumped after trying to ticket driverless car for illegal U-turn', 'https://www.theguardian.com/technology/2025/sep/29/waymo-illegal-u-turn-driverless-car', 'Technology', '2025-09-30T01:12:02Z', 'San Bruno officers pull over Waymo but say a ticket wasn’t issued, as ‘citation books don’t have a box for “robot”’', 'If a driver makes an illegal U-turn, but no one is behind the wheel, does the car still get a ticket? A police department in  California  grappled with this existential question last week.   During a DUI enforcement operation, officers in San Bruno pulled over a car without anyone behind the wheel after the autonomous vehicle made an illegal U-turn at a light. A  post  by the San Bruno police department on Saturday shows an officer looking into a  Waymo  – the leading autonomous ride-hailing vehicle in the  San Francisco  Bay Area – after stopping the signature white car.   “Since there was no hu

In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

print("Ping:", es.ping())
print("Info:", es.info().body)  # .info() now returns an ApiResponse object



Ping: True
Info: {'name': 'ab8514cce171', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'GMVXH0jPSN6Y_3VSCJWBSw', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [4]:
from elasticsearch import Elasticsearch

# connect to your cluster
es = Elasticsearch("http://localhost:9200")

INDEX_NAME = "paperboy"

# 1. Create index (if not exists)
def create_index_if_not_exists(dim=1024):
    if not es.indices.exists(index=INDEX_NAME):
        es.indices.create(
            index=INDEX_NAME,
            body={
                "mappings": {
                    "properties": {
                        "id": {"type": "keyword"},
                        "title": {"type": "text"},
                        "url": {"type": "keyword"},
                        "section": {"type": "keyword"},
                        "published_at": {"type": "date"},
                        "trail_text": {"type": "text"},
                        "body": {"type": "text"},
                        # BM25 works automatically on text fields
                        "embedding": {"type": "dense_vector", "dims": dim}
                    }
                }
            }
        )
        print(f"Index {INDEX_NAME} created")
    else:
        print(f"Index {INDEX_NAME} already exists")

# 2. Upsert document
def upsert_article(row: dict):
    doc_id = row["id"]  # use Guardian's id as unique key
    es.update(
        index=INDEX_NAME,
        id=doc_id,
        body={
            "doc": row,
            "doc_as_upsert": True
        }
    )
    print(f"Upserted article {doc_id}")


In [ ]:
def row_to_doc(row):
    return {
        "id": row[0],
        "title": row[1],
        "url": row[2],
        "section": row[3],
        "published_at": row[4],
        "trail_text": row[5],
        "body": row[6],
        "embedding": row[7],  # already a list of floats
    }

In [6]:
for row in all_articles:
    upsert_article(row_to_doc(row))

Upserted article australia-news/live/2025/oct/03/png-agreement-aukus-submarines-pact-anthony-albanese-richard-marles-retired-admiral-slams-aukus-deal-gaza-flotilla-intercepted-ntwnfb
Upserted article australia-news/2025/oct/03/further-blueberries-testing-shows-no-traces-of-banned-chemical-thiometon-nsw-authorities-say
Upserted article music/2025/oct/03/taylor-swift-the-life-of-a-showgirl-review
Upserted article commentisfree/2025/oct/03/the-irish-language-is-a-joy-not-a-burden-in-what-other-tongue-is-a-penis-a-wild-carrot
Upserted article film/2025/oct/03/bossing-it-the-makers-of-springsteen-deliver-me-from-nowhere-on-turning-jeremy-allen-white-into-the-rock-icon
Upserted article us-news/2025/oct/03/my-travels-through-chicago-tear-gas-resistance-and-trumps-big-immigration-crackdown
Upserted article tv-and-radio/2025/oct/03/the-simpsons-has-been-in-decline-as-long-as-ive-been-alive-why-its-time-for-the-show-to-end-for-good
Upserted article law/2025/oct/03/trumps-crackdown-on-lawyers-sho

In [10]:
res = es.search(
    index=INDEX_NAME,
    query={"multi_match": {"query": "climate change", "fields": ["title", "body"]}}
)
print(res.get('hits').get('hits')[0]["_score"])

11.714782
